In [4]:
!pip install opencv-python

In [5]:
!pip install tensorflow

In [5]:
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import numpy as np
import os
import pandas as pd
import cv2
import random

# Function to apply random transformation to an image and a point
def apply_random_transformation(image, point):
    # List to store the transformed images and corresponding points
    transformed_images = []
    transformed_points = []

    # Loop through each possible flip (no flip, horizontal flip, vertical flip, both flips)
    for flip_code in [0, 1, -1]:
        aug_point = point.copy()
        aug_image = cv2.flip(image, flip_code)
        if flip_code in [-1, 0]:
            aug_point[0] = image.shape[1] - point[0]
        if flip_code in [-1, 1]:
            aug_point[1] = image.shape[0] - point[1]
        
        transformed_images.append(aug_image)
        transformed_points.append(aug_point)
    #Uncomment this to add more augmented data, that takes flips and applies small rotations (between -30 and 30 degrees)
    """
    # Data with more change (rotations, noise added)
    for flip_code in [0, 1, -1, 2, random.choice([-1, 0, 1, 2]), random.choice([-1, 0, 1, 2])]:
        aug_point = point.copy()
        if(flip_code == 2):
            aug_image = image.copy()
        else:
            aug_image = cv2.flip(image, flip_code)
            
        if flip_code in [-1, 0]:
            aug_point[0] = image.shape[1] - point[0]
        if flip_code in [-1, 1]:
            aug_point[1] = image.shape[0] - point[1]
            
        angle = random.uniform(-30, 30)
        center = (aug_image.shape[1] // 2, aug_image.shape[0] // 2)
        rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)

        aug_image = cv2.warpAffine(aug_image, rotation_matrix, (aug_image.shape[1], aug_image.shape[0]), flags=cv2.INTER_NEAREST)
        aug_point = np.dot(np.array(point) - center, rotation_matrix[:, :2].T) + np.array(center)

        # Append the transformed image and point to the lists
        transformed_images.append(aug_image)
        transformed_points.append(aug_point.astype(int))
    """
    return transformed_images, transformed_points

# Set the path to your dataset
dataset_path_plume = './images/plume'
dataset_path_nonplume = './images/no_plume'

# Set the path to the directory where augmented images will be saved
augmented_path_plume = './images/plume_new'
augmented_path_nonplume = './images/no_plume_new'

# path for the CSV file to store point locations
csv_path = './metadata_aug.csv'

# Read the dataset CSV file
metadata = pd.read_csv(csv_path)


# Loop through each image file and perform data augmentation
for dataset_path in [dataset_path_plume, dataset_path_nonplume]:
    for idx, img_file in enumerate(os.listdir(dataset_path)):
        if img_file.endswith('.tif'):

            img_path = os.path.join(dataset_path, img_file)
            # Open the TIFF image using PIL
            img = Image.open(img_path)
            img_array = np.array(img)

            # Retrieve the corresponding point location from the dataset
            point_info = metadata[(metadata['id_coord'] == img_file[-11:-4]) & (metadata['date'] == int(img_file[0:8]))]
            x_original, y_original = point_info['coord_x'].values[0], point_info['coord_y'].values[0]


            # Apply random transformation to the image and point
            transformed_images, transformed_points = apply_random_transformation(img_array, [x_original, y_original])
            # Save augmented images and update metadata
            for i, (transformed_image, transformed_point) in enumerate(zip(transformed_images, transformed_points)):
                # Save the image
                augmented_filename = f"{img_file[:-4]}_aug_{i}.tif"
                augmented_filepath = os.path.join(dataset_path + "_new", augmented_filename)
                cv2.imwrite(augmented_filepath, transformed_image)

                # Create a DataFrame with the augmented data
                augmented_data = pd.DataFrame({
                    'date': point_info['date'],
                    'id_coord': augmented_filename[-17:-10],
                    'plume': point_info['plume'],
                    'set': "augmented",
                    'lat': point_info['lat'],
                    'lon': point_info['lon'],
                    'coord_x': transformed_point[0],
                    'coord_y': transformed_point[1],
                    'path': augmented_filepath[2:-4],
                })

                # Concatenate the augmented data with the existing metadata
                metadata = pd.concat([metadata, augmented_data], ignore_index=True)
                
# Save the updated metadata to the existing CSV file
metadata.to_csv(csv_path, index=False)

In [4]:
metadata.to_csv(csv_path, index=False)